# Lesson 4: Working with APIs

In this assignment, you'll be performing basic sentiment analysis on a Twitter data stream using Twitter's developer APIs.

### What is an API?
An [Application Programming Interface](https://en.wikipedia.org/wiki/Application_programming_interface) (API) is an interface or communication protocol between a client and a server intended to simplify the building of client-side software. In other words, it allows you to bypass an application's typical user interface (like Twitter's timeline) and take in data directly or perform functions in a more developer friendly manner.

For example, if you wanted to save the text from 500 tweets on your timeline without using Twitter's APIs, you would have to scroll through your timeline and copy and paste each tweet into a csv file. With Twitter's APIs, you can write a function in Python that will listen for new tweets and push the text directly into your favorite data structure.

### Getting Started
In order to use the Twitter streaming API you'll need a developer account, which you can apply for [here](https://developer.twitter.com/). This process may take some time (even a few days), so start early!

Once you have that set up, you'll need to create an Application using the [Application Console](https://developer.twitter.com/en/apps). This will get you all the credentials you need to start streaming Twitter data. Just fill in the required values and you're ready to go.

In the application console, click on "Keys and Access Tokens" to get your credentials. Under "Application Settings" you should be able to find:
* Consumer API Keys
    * API Key
    * API Secret

Fill those values into the fields below under `TWITTER_APP_KEY` and `TWITTER_APP_SECRET` respectively.

Next, go to the section in the application console labelled "Your Access Token", and click "Create my access token". You should see the following two values:
* Access token & access token secret
    * Access Token
    * Access Token Secret

Fill those values into the fields below under `TWITTER_KEY` and `TWITTER_SECRET` respectively.


In [ ]:
TWITTER_APP_KEY = ""
TWITTER_APP_SECRET = ""
TWITTER_KEY = ""
TWITTER_SECRET = ""

The most popular Python client for the Twitter streaming API is [Tweepy](https://github.com/tweepy/tweepy), which we will be using here. Installation is as simple as running `pip install tweepy` in a terminal.

While you're at it, `pip install dataset` as well. [dataset](https://dataset.readthedocs.io/en/latest/index.html) is a package used to make working with databases in Python much easier, and we'll be using it here.

And finally, `pip install textblob`. [textblob](https://textblob.readthedocs.io/en/dev/) is a Natural Language Processing library we'll be using to do sentiment analysis on the tweets we collect.

Once you've installed Tweepy and written down your credentials, you're ready to get started.

Let's import the libraries we'll need for working with the data.

In [ ]:
import tweepy
import dataset
from textblob import TextBlob

`tweepy`, `dataset`, and `textblob` are the packages we just installed, so we'll need them later.

We'll now get authenticated with Twitter's API, which Tweepy handles for us as follows.

In [ ]:
auth = tweepy.OAuthHandler(TWITTER_APP_KEY, TWITTER_APP_SECRET)
auth.set_access_token(TWITTER_KEY, TWITTER_SECRET)

Then, we'll create an api object to pull data from Twitter. Tweepy also handles this for us like this:

In [ ]:
api = tweepy.API(auth)

You can fill in whatever terms you would like to track below. We will only receive tweets containing these terms from the stream. The connection string specifies the name of the database file we'll be storing the tweets in, and the table name specifies the name of the table within that database that stores the tweets. The csv name we'll use later to write the tweets to a csv file.

In [ ]:
TRACK_TERMS = ["", "", "", ""]
CONNECTION_STRING = "sqlite:///tweets.db"
CSV_NAME = "tweets.csv"
TABLE_NAME = "tweets"

Let's go ahead and create a database object. dataset's `connect` function will connect to an existing database of the specified name in the project directory if it exists, or create one if it does not.

In [ ]:
db = dataset.connect(CONNECTION_STRING)

We have a little more setting up to do before we can start streaming from the Twitter Streaming API. Tweepy requires a set of callback functions bundled into a `StreamListener` class that it will call every time a tweet comes in, so that it knows how we want it to handle successful and unsuccessful api calls. We'll set this up as follows: 

In [ ]:
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):
        if hasattr(status, "retweeted_status"):
            return
        if hasattr(status, "quoted_status"):
            return
        if db[TABLE_NAME] and len(db[TABLE_NAME]) >= 10000:
            # disconnect from stream when we have enough data
            return False
#         description = status.user.description
        text = status.text
        if hasattr(status, "extended_tweet"):
            text = status.extended_tweet["full_text"]
        print(len(db[TABLE_NAME]), text)
        name = status.user.screen_name
#         user_created = status.user.created_at
        followers = status.user.followers_count
        id_str = status.id_str
        created = status.created_at
#         retweets = status.retweet_count
        blob = TextBlob(text)
        sent = blob.sentiment
            
        table = db[TABLE_NAME]
        
        # insert whatever fields are important to you below
        try:
            table.insert(dict(
#                 user_description=description,
                text=text,
                user_name=name,
#                 user_created=user_created,
                user_followers=followers,
#                 id_str=id_str,
                created=created,
#                 retweet_count=retweets,
                polarity=sent.polarity,
                subjectivity=sent.subjectivity,
            ))
        except Exception as err:
            print(err)

    def on_error(self, status_code):
            if status_code == 420:
                return False

This may look a bit complicated but the bulk of the code is set up for you. Here we simply take in a tweet and record the relevant attributes to our database. We also do the sentiment analysis on the text of the tweets here. Anything you'd like to change will probably fall under the `on_status` function. The twitter developer docs have more specifics about the fields available to you within the status object.

Now for the fun part.

### Streaming Tweets

We'll start by instantiating a stream listener and a tweepy stream. The tweepy stream requires the stream listener object we created just now as well as the api object we created earlier with our dev credentials. Finally, we ask the stream to filter for all tweets that contain our `TRACK_TERMS` and we immediately start to see tweets come in!

In [ ]:
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(track=TRACK_TERMS)


### Storing the data

It's difficult to process the tweets as they're coming in, so it will benefit you to store them in a more permanent form on disk so you can work with a static set of data instead of one that is constantly changing. We've already done this in the `on_status` function by storing the tweets in a database, but we can also save it in the more familiar csv format so we can open it in other tools like Excel. Keep in mind that once the dataset gets large enough it may be more difficult to work with a csv than a database, but for this example either should be fine.

In [ ]:
import datafreeze

result = db["tweets"].all()
datafreeze.freeze(result, format='csv', filename=CSV_NAME)


### Visualize
By now you've probably gotten used to visualizing data, so feel free to pick your favorite kind of visualization and present the data in a digestible way.